In [ ]:
import os
import json
import time
import random
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# Aseguramos que existan las carpetas antes de guardar
os.makedirs('../config/logs', exist_ok=True)
os.makedirs('../data/csv', exist_ok=True)

current_time = datetime.now().strftime('%Y_%m_%d-%Hh_%Mm')
logs_path = f"../config/logs/logs_{current_time}.log"
csv_path = f"../data/csv/data_{current_time}.csv"

#FINANCITAL_API = 'https://www.kavak.com/api/vip-ui/mx/calculator/468814?upfront-amount=116499'
FINANCITAL_API = 'https://www.kavak.com/api/vip-ui/mx/calculator'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9'
}


def get_minimum_upfront_amount(price):
    return float(price) * 0.16

In [ ]:
def log_writer(message):
    current_time = datetime.now().strftime('%Y_%m_%d-%Hh_%Mm')
    try:
        with open(logs_path, 'a', encoding='utf-8') as f:
            f.write(f"{current_time} " + message + '\n')
    except Exception as e:
        print("No se pudo escribir el log: {e}")

In [ ]:
def api_requester(auto_id, headers, price):
    api_url = f"{FINANCITAL_API}/{auto_id}"
    upfront_amount = int(get_minimum_upfront_amount(price))
    query_params = {
        'upfront-amount': upfront_amount
    }
    
    try:
        response = requests.get(api_url, headers=headers, params=query_params)
        response.raise_for_status()
        return response

    except requests.exceptions.RequestException as e:
        #print(f"Error conectando con auto {auto_id}: {e}")
        log_writer(f"Error conectando con auto {auto_id}: {e}")
    
        return None

In [ ]:
def get_plan_info(plan, auto_id):
    try:
        mensualidades = plan['installments']
        enganche = plan['value']
        tasa = plan['rate']
        if plan['insurance']:
            seguro = plan['insurance']['installmentAmount']
        else: seguro = None
        return mensualidades, enganche, tasa, seguro
    except Exception as e:
        #print(f"Sucedio un error extrayendo los planes del auto: {auto_id}: {e}")
        log_writer(f"Sucedio un error extrayendo los planes del auto: {auto_id}: {e}")
        return None

In [ ]:
def get_upfront_info(inputData, auto_id):
    try:
        value = inputData['value']
        min_upfront_value = inputData['min']
        max_upfront_value = inputData['max']
        return value, min_upfront_value, max_upfront_value
    except Exception as e:
        #print(f"Sucedio un error extrayendo los enganches del auto: {auto_id}: {e}")
        log_writer(f"Sucedio un error extrayendo los enganches del auto: {auto_id}: {e}")
        return None

In [ ]:
def extract_financial_info(auto_id, paymentPlans, inputData, price):
    data_plan_list = []
    value, min_upfront_value, max_upfront_value = get_upfront_info(inputData, auto_id)

    for plan in paymentPlans:
        plazo, mensualidad, tasa, seguro = get_plan_info(plan, auto_id)
        data_dict = {
            'ID_Auto':auto_id,
            'Precio':price,
            'Plazo':plazo,
            'Mensualidad':mensualidad, 
            'Tasa':tasa, 
            'Seguro':seguro,
            'Enganche_Simulado':value, 
            'Enganche_Min':min_upfront_value, 
            'Enganche_Max':max_upfront_value
            }

        data_plan_list.append(data_dict)

    return data_plan_list

In [ ]:
def save_batch_to_csv(batch_data, path):
    if not batch_data:
        return
    
    try:
        df = pd.DataFrame(batch_data)
        write_headers = not os.path.exists(path)
        df.to_csv(path, mode='a', index=False, header=write_headers, encoding='utf-8-sig')
        print(f"Batch de {len(batch_data)} filas guardado.")
    except Exception as e:
        #print(f"Error al intentar guardar los datos del batch: {e}")
        log_writer(f"Error al intentar guardar los datos del batch: {e}")

In [ ]:
def main():
    batch_buffer = []
    BATCH_SIZE = 20

    with open('../data/dataset_autos.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            car = json.loads(line)
            id = car['id']
            slug = car['slug']
            price = car['price']

            print(f"{i} Extrayendo datos para el ID: {car['id']}, {car['slug']}, {car['price']}")
   
            referer = car['slug'] + "?" + "id=" + car['id']
            headers_copy = HEADERS.copy()
            headers_copy['Referer'] = referer

            time.sleep(random.uniform(1.5, 4))
            response = api_requester(car['id'], headers_copy, car['price'])
            
            if response is None:
                log_writer(f"No se obtuvo respuesta para el auto con ID: {id}")
                continue
            data_json = response.json()

            if 'offers' in data_json:
                try:
                    paymentPlans = data_json['offers']['paymentPlan']['paymentOptions']['UPFRONT_VALUE']
                    inputData = data_json['offers']['inputData']

                    planes_extraidos = extract_financial_info(car['id'], paymentPlans, inputData, car['price'])
                    batch_buffer.extend(planes_extraidos)
                except Exception as e:
                    log_writer(f"No se encontro un llave para el carro: {car['id']}")    
                    continue

            else: 
                print(f"Auto no disponible {car['id']}")
                log_writer(f"Auto no disponible {car['id']}")
                continue
    
            if (i % BATCH_SIZE) == 0:
                print(f"Buffer lleno ({len(batch_buffer)} registros). Guardando batch...")
                save_batch_to_csv(batch_buffer, csv_path)
                batch_buffer = []
            
        if batch_buffer:
            print("Guardando últimos registros pendientes...")
            save_batch_to_csv(batch_buffer, csv_path)
            
            print("✨ Proceso Terminado.")

In [ ]:
if __name__ == '__main__':
    main()